In [ ]:
# import necessary modules

import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras import layers

In [ ]:
# import dataset from directory (what is cleaning of dataset ?? )

import pathlib

data_dir = "C:\\Abhishek_Data\\My_Data\\Datasets\\Classification\\PetImages"
data_dir = pathlib.Path(data_dir)

batch_size = 16
img_width = 227
img_height = 227

train_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=123
)

val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=123
)

class_names = train_ds.class_names
print(class_names)

In [ ]:
# data augmentation

num_classes = len(class_names)

data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomCrop(height=img_height, width=img_width)
  ]
)

In [ ]:
# create the model

model = Sequential()
model.add(data_augmentation)
#model.add(layers.Rescaling(1./255))

model.add(layers.Conv2D(96, 11, strides=4, padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(3,3), strides=2))

model.add(layers.Conv2D(256, 5, strides=1, padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(3,3), strides=2))

model.add(layers.Conv2D(384, 3, strides=1, padding="same"))
model.add(layers.Conv2D(384, 3, strides=1, padding="same"))

model.add(layers.Conv2D(256, 3, strides=1, padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(3,3), strides=2))

model.add(layers.Flatten())

model.add(layers.Dense(4096))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.4))

model.add(layers.Dense(4096))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.4))

model.add(layers.Dense(num_classes))
model.add(layers.BatchNormalization())
model.add(layers.Activation("softmax"))

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

model.summary()

In [ ]:
# model training
import datetime, os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tensorboard_callback]
)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('cat_dog_alexnet_scratch.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# save as keras (h5) -> works with 667 MB of file
model.save("cat_dog_alexnet.h5")

In [ ]:
# save as tf SavedModel (pb) -> does not work with 3 MB of file
alexnetNet_save_path = "C:\\Abhishek_Data\\My_Data\\VSC_Workspace\\Python\\classification\\saved_model_alexnet_scratch"
tf.saved_model.save(model, alexnetNet_save_path)